In [1]:
import requests
import web3
import pandas as pd

import libs.common.utils as utils
import libs.aave.api as aave_api
import libs.common.payload as payload
import libs.common.etherscan as es
import eth_abi
import config.config as cfg

In [2]:
ws = web3.Web3(web3.Web3.HTTPProvider(cfg.config["http_url"]))

In [3]:
contract_address = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"

In [4]:
proxy_addr = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
impl_addr = "0x085E34722e04567Df9E6d2c32e82fd74f3342e79"
pool_contract = es.get_contract(proxy_addr,impl_addr)

In [5]:
res = pool_contract.functions.getReservesList().call(block_identifier=18000000)

In [6]:
pool_contract.functions.getUserAccountData("0x1111567E0954E74f6bA7c4732D534e75B81DC42E").call()

[40975917587262136813643,
 33644389432055738516308,
 160742577435524354947,
 8600,
 8250,
 1047404625850338923]

In [13]:
def parse_function_abi(contract_abi):
    function_abi = {}
    for member in contract_abi:
        if member["type"] == "function":
            inputs = {(inp["name"] if inp["name"] != "" else "value"):inp["type"] for inp in member["inputs"]}
            outputs = {(out["name"] if out["name"] != "" else "value"):out["type"] for out in member["outputs"]}
            function_abi[member["name"]] = {"inputs":inputs,"outputs":outputs}
    return function_abi

In [15]:
function_abi = parse_function_abi(pool_contract.functions.abi)

In [23]:
def function_call(function_name, params, block_number):
    inputs = function_abi[function_name]["inputs"]
    outputs = function_abi[function_name]["outputs"]

    val = payload.func_call_payload(0, contract_address, function_name, inputs, params, block_number)
    resp = requests.post(cfg.config["http_url"], json=val)
    
    values = eth_abi.decode(outputs.values(), bytes.fromhex(resp.json()["result"][2:]))
    keys = list(outputs.keys())
    result = {keys[i]: values[i] for i in range(len(keys))}
    return result

In [25]:
function_name = "getUserAccountData"
params = {
    "user":"0x1111567E0954E74f6bA7c4732D534e75B81DC42E"
}
block_number = hex(18670000)
function_call(function_name, params, block_number)

{'totalCollateralETH': 40975140846076479787939,
 'totalDebtETH': 33480094817366720358262,
 'availableBorrowsETH': 324396380646375466788,
 'currentLiquidationThreshold': 8600,
 'ltv': 8250,
 'healthFactor': 1052524531960013249}

In [26]:
function_name = "getReservesList"
params = {}
block_number = hex(18670000)
function_call(function_name, params, block_number)

{'value': ('0xdac17f958d2ee523a2206206994597c13d831ec7',
  '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
  '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
  '0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e',
  '0xe41d2489571d322189246dafa5ebde1f4699f498',
  '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
  '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9',
  '0x0d8775f648430679a709e98d2b0cb6250d2887ef',
  '0x4fabb145d64652a948d72533023f6e7a623c7c53',
  '0x6b175474e89094c44da98b954eedeac495271d0f',
  '0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c',
  '0xdd974d5c2e2928dea5f71b9825b8b646686bd200',
  '0x514910771af9ca656af840dff83e8264ecf986ca',
  '0x0f5d2fb29fb7d3cfee444a200298f468908cc942',
  '0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2',
  '0x408e41876cccdc0f92210600ef50372656052a38',
  '0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f',
  '0x57ab1ec28d129707052df4df418d58a2d46d5f51',
  '0x0000000000085d4780b73119b644ae5ecd22b376',
  '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
  '0xd533a949740bb3306d119cc777